### Przygotwanie danych

In [ ]:
# pobierzmy ponownie zbiór
import urllib.request
import os
import pandas as pd
import matplotlib.pyplot as plt

plik = 'GDP_happiness.csv'
URL = "https://byes.pl/wp-content/uploads/datasets/" + plik
if not os.path.isfile(plik):
    print('Pobieram plik z ', URL)
    urllib.request.urlretrieve(URL, plik)
    print('Pobrano plik')
else:
    print(f'Plik {plik} już jest na dysku')

dane = pd.read_csv(plik, index_col=[0])
dane = dane.fillna(dane.mean(axis=0))
dane.tail()

display(dane)

X = dane['GDP per capita'].values
Y = dane['happiness'].values

### Metody z biblioteki [Scikit-learn](https://scikit-learn.org/stable/)

Metody uczenia maszynowego, czy to do klasyfikacji, regresji, grupowania, czy dopasowywania rozkładów do danych, dostępne w bibliotece Scikit-learn nazywane są - nie wiedzieć czemu - *estymatorami*. <br>
Sposób korzytania z każego estymatora został ujednolicony. Oto przykład regresji liniowej.

In [ ]:
from sklearn.linear_model import LinearRegression

2. Utworzenie instancji

In [ ]:
model_lin = LinearRegression()

3. Dopasowanie *estymatora* do danych metodą ```fit```: patrz komentarz pod algorytmem (4).

In [ ]:
model_lin.fit(X.reshape(-1,1), Y)

4. Obliczenia/predykcje z wykorzystaniem metody ```predict``` *estymatora*

In [ ]:
X_test = np.linspace(start=X.min(), stop=X.max(), num=300)
Y_pred = model_lin.predict(X_test.reshape(-1,1))

plt.scatter(X,Y, alpha=0.7)
plt.plot(X_test, Y_pred, color='tab:orange', linewidth=3)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Można też sprawdzić wartości parametrów otrzymanego modeli liniowego.

In [ ]:
print(f'Parametry modelu liniowego: a = {np.round(model_lin.coef_,5)}, b = {np.round(model_lin.intercept_,5)}')

Oto inne ważne czynności, które wykonuje się w ramach dopasowywania modelu do danych.

Podział zbioru na część do trenowania i testowania

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

Wskaźnik jakości modelu

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(Y, model_lin.predict(X.reshape(-1,1)))

Oto **przykład** demonstrujący jednolitość obsługi *estymatorów* biblioteki Scikit-learn

In [ ]:
X = dane['GDP per capita'].values
Y = dane['happiness'].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# =========  Model liniowy =========
from sklearn.linear_model import LinearRegression
model_lin = LinearRegression()
model_lin.fit(X_train.reshape(-1,1), 
              Y_train)
print(f'Parametry modelu liniowego: {np.round(model_lin.coef_,5)}, {np.round(model_lin.intercept_,5)}')
MSE_lin = mean_squared_error(Y_test, model_lin.predict(X_test.reshape(-1,1)))
print(f'Błąd średniokwadratowy modelu liniowego: {MSE_lin:0.3}\n')

# ===== Uogólniony model liniowy (ang. Generalized Linear Model) =====
from sklearn.preprocessing import PolynomialFeatures
model_GLM = LinearRegression()
gen_features = PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)
model_GLM.fit(gen_features.fit_transform(X_train.reshape(-1,1)), 
              Y_train)
print(f'Parametry modelu GLM: {np.round(model_GLM.coef_,4)}, {np.round(model_GLM.intercept_,5)}')
MSE_GLM = mean_squared_error(Y_test, model_GLM.predict(gen_features.fit_transform(X_test.reshape(-1,1))))
print(f'Błąd średniokwadratowy modelu GLM: {MSE_GLM:0.3}\n')

#==== Maszyna wektorów wspierających (ang. Support Vector Machine) ====
# SVR dla regresji, SVC dla klasyfikacji
from sklearn.svm import SVR
model_svr = SVR(kernel='rbf', gamma='scale', C=1)
model_svr.fit(X_train.reshape(-1,1), 
              Y_train)
MSE_SVR = mean_squared_error(Y_test, model_svr.predict(X_test.reshape(-1,1)))
print(f'Błąd średniokwadratowy modelu SVR: {MSE_SVR:0.3}')

# Predykcje wszystkich modeli dla całego zakreso osi X
os_x = np.linspace(start=X.min(), stop=X.max(), num=300)
y_lin_pred = model_lin.predict(os_x.reshape(-1,1))
y_GLM_pred = model_GLM.predict(gen_features.fit_transform(os_x.reshape(-1,1)))
y_svr_pred = model_svr.predict(os_x.reshape(-1,1))

# Wizualizacja
plt.figure(figsize=(10,7))
plt.scatter(X_train, Y_train, label='dane treningowe', alpha=0.7)
plt.scatter(X_test, Y_test, edgecolor='black', facecolor='none', label='dane testujące')
plt.plot(os_x, y_lin_pred, label='model liniowy', color='tab:orange')
plt.plot(os_x, y_GLM_pred, label=f'model GLM', color='tab:red')
plt.plot(os_x, y_svr_pred, label='model SVR', color='tab:green')
plt.xlabel(dane.columns[0], fontsize=14)
plt.ylabel(dane.columns[1], fontsize=14)
plt.legend(fontsize=12, shadow=True, loc='lower right')
plt.ylim([Y.min()-0.1, Y.max()+0.5])
plt.show()

**Zastanów się** <br>
Wykonaj wielokrotne dopasowania modeli do różnych podziałów zbioru danych na część do trenowania i do testowania. Wyciągnij wnioski. <br>
Jak oceniasz adekwatność każdego z modeli, tzn. czy poprawnie opisuje tendencje obserwowane w danych? <br>
Jaki kolejny model proponujesz rozważyć? <br>
Jak można ocenić adekwatność modelu w przypadku danych wielowymiarowych?